In [41]:
from Softs.Spanve import *
import os

save_dir = './recode/IMPUTEDLIBD/'
data_dirs = './data/SpatialLIBD/'
data_ids = [file for file in os.listdir(data_dirs) if file.endswith('.h5ad')]

import time
import warnings
warnings.filterwarnings('ignore')
for data_id in data_ids:
    print(time.ctime(),data_id)
    adata = sc.read_h5ad(os.path.join(data_dirs,data_id))
    row_counts = adata.X 
    pre_adata = adata_preprocess(adata)
    
    for preprocessed_int in [False,True]:
        
        if preprocessed_int:
            adata = adata_preprocess_int(adata)
        
        model = Spanve(adata,verbose=True)

        model.fit()
        
        Xi = model.impute_from_graph(X=pre_adata.X[:,model.rejects])
        model.adata.obsm['spanve.imputed'] = Xi
        model.adata.obsm['spanve.imputed.pca'] = sc.pp.pca(Xi)
        
        model.adata.X = row_counts
        
        model.adata.write_h5ad(
            os.path.join(save_dir,'processed',f"{data_id}{['','_pre'][preprocessed_int]}")
        )
        
# merge the imputed data from same data
for data_id in data_ids:
    adata1 = sc.read_h5ad(os.path.join(save_dir,f"{data_id}"))
    adata2 = sc.read_h5ad(os.path.join(save_dir,f"{data_id}_pre"))
    adata1.var.columns = ['n_cells','spanve_spatial_features']
    adata1.var['spanve_spatial_features_pre'] = adata2.var['span_spatial_features']
    adata1.obsm['spanve.imputed.pre'] = adata2.obsm['spanve.imputed']
    adata1.obsm['spanve.imputed.pca.pre'] = adata2.obsm['spanve.imputed.pca']
    adata1.write_h5ad(os.path.join(save_dir,f"{data_id}"))